In [1]:
import tensorflow as tf
import numpy as np
import os, re
from tensorflow.keras.models import load_model
from tensorflow import keras
from PIL import Image, ImageOps

# 이미지 데이터를 Average Hash로 변화하기
def average_hash(fname,size=16):
    fname2 = fname[len(search_dir):]
    
    #이미지 캐시하기
    cache_file = cache_dir + "\\" + fname2.replace('\\','_') + ".csv"
    if not os.path.exists(cache_file):  # 해시 생성하기
        img = Image.open(fname)
        img = img.convert('L').resize((size, size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt="%.0f", delimiter=",")
    else: 
        px = np.loadtxt(cache_file, delimiter=",")
    return px

#해밍 거리 구하기
def hamming_dist(a,b):
    aa = a.reshape(1,-1)      # 1차원 배열로 변환
    ab = b.reshape(1,-1)
    dist = (aa != ab).sum()
    return dist

# 모든 폴더의 이미지 파일에 처리 적용하기
def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root,f)
            if re.search(r'\.(jpg|jpeg|png)$',fname):
                yield fname

def fine_image(fname, rate):
    src = average_hash(fname)

    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst) / 256
        #print("[check]",fname)
        if diff_r < rate:
            yield (diff_r,fname)

In [2]:
from flask import Flask, redirect, url_for, request, Response, render_template
from flask import send_file

app = Flask(__name__)
@app.route('/', methods = ['POST', 'GET'])
def home():
    return render_template('avhash-search-output.html')

#자바 서블릿단에서 값을 받아오는 플라스크 서버 함수
@app.route('/painting', methods = ['POST', 'GET'])
def painting():
    filename = request.args['filename'] 
    
@app.route('/draw', methods = ['POST', 'GET'])
def draw():
    best_model = load_model('./model/Tattoo_10_Model_002_1.0000.hdf5')
    tattoo_dir_list = ['고래', '꽃', '나비', '달', '십자가', '하트']
    
    PATH = './엣지엣지/butterfly_72.jpg'

    img = Image.open(PATH).resize((224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = best_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    #print(score)
    """print(
        "\nThis image most likely belongs to {} with a {:.2f} percent confidence."
        .format(tattoo_dir_list[np.argmax(score)], 100 * np.max(score))
    )"""

    fold_name = tattoo_dir_list[np.argmax(score)]

    # 파일 경로 지정하기
    search_dir = './추려추려/' + fold_name
    cache_dir = './추려추려/cache_avhash'
    
    if not os.path.exists(cache_dir):
        os.mkdir(cache_dir)

    # 찾기
    #srcfile = search_dir + "\\whale_28.jpg"
    srcfile = PATH
    html = ""
    sim = list(fine_image(srcfile, 1.5))
    sim = sorted(sim[:5], key=lambda x:x[0])

    image_addr = []
    # 비슷한 이미지를 찾은 결과(해밍거리 0.25 미만)를 이렇게 html파일로 만들어서 저장
    for r,f in sim:
        print(r,">",f)
        s = '<div style="float:left;"><h3>[차이 :' + str(r) + '-' + \
            os.path.basename(f) + ']</h3>' + \
            '<p><a href= "' + f + '"><img src="'+f+'"width=400>'+ \
            '</a></p></div>'
        html += s
        image_addr.append(f)

    #HTML로 출력
    html = """<html><head><meta charset="utf8"></head> 
    <body><h3>원래 이미지</h3><p>
    <img src='{0}' width=400></p>{1}
    </body></html>""".format(srcfile, html)

    with open("./templates/avhash-search-output.html", "w", encoding="utf-8") as f:
        f.write(html)
    
    return render_template('avhash-search-output.html', image_file = image_addr)

if __name__ == "__main__":
    app.run(host="localhost", port="9000")   

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


In [3]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\sm120\anaconda3\envs\deep\lib\site-packages (2.3.1)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
best_model = load_model('./model/Tattoo_10_Model_002_1.0000.hdf5')
tattoo_dir_list = ['고래', '꽃', '나비', '달', '십자가', '하트']
PATH = './엣지엣지/cross_3001.jpg'

img = Image.open(PATH).resize((224, 224))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = best_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

#print(score)
"""print(
    "\nThis image most likely belongs to {} with a {:.2f} percent confidence."
    .format(tattoo_dir_list[np.argmax(score)], 100 * np.max(score))
)"""

fold_name = tattoo_dir_list[np.argmax(score)]
# 파일 경로 지정하기
search_dir = './templates/' + fold_name
search_dir = os.getcwd() + "\\추려추려\\" + fold_name
cache_dir = os.getcwd() + "\\추려추려\\cache_avhash"
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

# 찾기
#srcfile = search_dir + "\\whale_28.jpg"
srcfile = PATH
html = ""
sim = list(fine_image(srcfile, 1.5))
sim = sorted(sim[:5], key=lambda x:x[0])

image_addr = []
# 비슷한 이미지를 찾은 결과(해밍거리 0.25 미만)를 이렇게 html파일로 만들어서 저장
for r,f in sim:
    print(f)
    s = '<div style="float:left;"><h3>[차이 :' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>' + \
        '<p><a href= "' + f + '"><img src="'+f+'"width=400>'+ \
        '</a></p></div>'
    html += s
    image_addr.append(f)

#HTML로 출력
html = """<html><head><meta charset="utf8"></head> 
<body><h3>원래 이미지</h3><p>
<img src='{0}' width=400></p>{1}
</body></html>""".format(srcfile, html)

with open("./templates/avhash-search-output.html", "w", encoding="utf-8") as f:
    f.write(html)

In [ ]:
@app.route('/get_image')
def get_image():
    if request.args.get('type') == '1':
        filename = 'ok.gif'
    else:
        filename = 'error.gif'
    return send_file(filename, mimetype='image/gif')

In [ ]:
from flask import Flask, redirect, url_for, request, Response
from flask import send_file

app = Flask(__name__)

@app.route('/draw/flower', methods = ['POST', 'GET'])
def draw():
    num = 3
    return send_file('./추려추려/butterfly_' + str(num) + '.jpg', mimetype='image/jpeg')

@app.route('/draw/heart', methods = ['POST', 'GET'])
def draw():
    #num = request.args["num"] 
    num = 3
    return send_file('./플라스크/butterfly_' + str(num) + '.jpg', mimetype='image/jpeg')

if __name__ == "__main__":
    app.run(host="localhost", port="9000")   

In [4]:
from flask import Flask, redirect, url_for, request, Response, render_template
from flask import send_file

app = Flask(__name__)

@app.route('/draw', methods = ['POST', 'GET'])
def draw():
    best_model = load_model('./model/Tattoo_10_Model_002_1.0000.hdf5')
    tattoo_dir_list = ['고래', '꽃', '나비', '달', '십자가', '하트']
    PATH = './엣지엣지/cross_1.jpg'

    img = Image.open(PATH).resize((224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = best_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    #print(score)
    """print(
        "\nThis image most likely belongs to {} with a {:.2f} percent confidence."
        .format(tattoo_dir_list[np.argmax(score)], 100 * np.max(score))
    )"""

    fold_name = tattoo_dir_list[np.argmax(score)]

    # 파일 경로 지정하기
    search_dir = './추려추려/' + fold_name
    cache_dir = './추려추려/cache_avhash'
    if not os.path.exists(cache_dir):
        os.mkdir(cache_dir)

    # 찾기
    #srcfile = search_dir + "\\whale_28.jpg"
    srcfile = PATH
    html = ""
    sim = list(fine_image(srcfile, 1.5))
    sim = sorted(sim[:5], key=lambda x:x[0])

    image_addr = []
    # 비슷한 이미지를 찾은 결과(해밍거리 0.25 미만)를 이렇게 html파일로 만들어서 저장
    for r,f in sim:
        print(r,">",f)
        s = '<div style="float:left;"><h3>[차이 :' + str(r) + '-' + \
            os.path.basename(f) + ']</h3>' + \
            '<p><a href= "' + f + '"><img src="'+f+'"width=400>'+ \
            '</a></p></div>'
        html += s
        image_addr.append(f)

    #HTML로 출력
    html = """<html><head><meta charset="utf8"></head> 
    <body><h3>원래 이미지</h3><p>
    <img src='{0}' width=400></p>{1}
    </body></html>""".format(srcfile, html)

    with open("./templates/avhash-search-output.html", "w", encoding="utf-8") as f:
        f.write(html)
    
    return render_template('avhash-search-output.html', image_file = image_addr)



# 페이지 이동 + 파일 업로드하기
#@app.route('/uploads/<filename>')
#def uploaded_file(filename):
#       filename = 'http://127.0.0.1:5000/uploads/' + filename
#       return render_template('template.html', filename = filename)




if __name__ == "__main__":
    app.run(host="localhost", port="9000")
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Nov/2020 16:18:28] "GET / HTTP/1.1" 404 -
[2020-11-24 16:18:30,950] ERROR in app: Exception on /draw [GET]
Traceback (most recent call last):
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SM107\anaconda3\envs\dl\lib\site-packages\flask\app.py", line 1936, i

In [ ]:
@app.route('/calculate', methods=['POST', 'GET'])
def calculate(num=None):
    ## 어떤 http method를 이용해서 전달받았는지를 아는 것이 필요함
    ## 아래에서 보는 바와 같이 어떤 방식으로 넘어왔느냐에 따라서 읽어들이는 방식이 달라짐
    if request.method == 'POST':
        #temp = request.form['num']
        pass
    elif request.method == 'GET':
        ## 넘겨받은 숫자 
        temp = request.args.get('num')
        temp = int(temp)
        ## 넘겨받은 문자
        temp1 = request.args.get('char1')
        ## 넘겨받은 값을 원래 페이지로 리다이렉트
        return render_template('submit_test.html', num=temp, char1=temp1)
    ## else 로 하지 않은 것은 POST, GET 이외에 다른 method로 넘어왔을 때를 구분하기 위함